## <font color=Green>GMD Table 5 </font>  
#### <font color=blue> *Frequency amplification factors in the year 2100 for the historic 1% average annual probability (100-year return period) extreme sea-level event at New York City* </font>


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr
import tarfile
#
import warnings
import matplotlib
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)
#
colrs = 'rcbgmyrkcbgm'

In [2]:
EXPDATE  = 230116
SCENARIO = ['ssp126','ssp585']
WORKFLO  = ["wf1f","wf2f","wf3f","wf4"]
YEAR     = ['2050','2100']
#
df_TEMP   = [] 
# open file
for scenario in SCENARIO:
    for wf in WORKFLO:
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/esl.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        tarFILE = 'esl.{arg1}.extremesealevel.extremesealevel.pointsoverthreshold.{arg2}_extremesl.tgz'.format(arg1=scenario,arg2=wf)
        file = tarfile.open(dataFOLDER+tarFILE)
        # extracting file
        file.extractall('./ESL_nc_data')
        file.close()
        #
        for year in YEAR:
            d_nc = xr.open_dataset('ESL_nc_data/' + 'esl.{arg1}.extremesealevel.extremesealevel.pointsoverthreshold.{arg2}_id12_year{arg3}_extremesl.nc'.format(arg1=scenario,arg2=wf,arg3=year))
            #
            percentList = [50, 17, 83]
            #
            af = [d_nc.amplification_factors_quantiles.values[49], d_nc.amplification_factors_quantiles.values[16] , d_nc.amplification_factors_quantiles.values[82]]
            af = np.around(af,2)
            #
            df_TEMP.append( [wf, scenario, year,] + af.tolist() )
df_ptile = pd.DataFrame( df_TEMP, columns=['Workflow', 'SSP','Year',] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
#
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} - {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( [ 'Workflow','SSP','Year', ] )[''] ).unstack().unstack().swaplevel(0, 1, axis=1)
#
display(df_ptile1)

Year                    2050                                     2100  \
                                                                        
SSP                   ssp126              ssp585               ssp126   
Workflow                                                                
wf1f      2.59 (1.80 - 4.18)  2.84 (1.94 - 4.65)  6.50 (3.15 - 17.15)   
wf2f      2.85 (1.92 - 4.80)  3.16 (2.09 - 5.43)  9.95 (4.10 - 33.78)   
wf3f      2.71 (1.86 - 4.33)  2.88 (1.98 - 4.70)  7.59 (3.73 - 19.56)   
wf4       2.94 (1.93 - 5.24)  3.41 (2.13 - 6.77)  9.92 (3.97 - 43.55)   

Year                                 
                                     
SSP                          ssp585  
Workflow                             
wf1f           22.07 (7.99 - 90.32)  
wf2f         55.65 (13.00 - 451.21)  
wf3f       145.48 (27.56 - 1629.17)  
wf4       126.43 (16.25 - 10283.39)